<br>

# Introdução

In [1]:
import os
import time
#import json
#import shutil
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileMerger, PdfFileReader

In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import DesiredCapabilities
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
import os
import sys
sys.path.append(os.path.expanduser('~/Documents/Sourcecode'))
from my_vault.credentials_eambiente import credential

<br>

## Create Driver

In [ ]:
def create_driver():
    # Escolhe o driver Firefox com Profile e Options
    profile = webdriver.FirefoxProfile()
    options = Options()
    options.headless = True
    global driver
    driver = webdriver.Firefox(firefox_profile=profile, options=options)
    
    # Add-Ons
    install_xpath = False
    if install_xpath:
        r = requests.get('https://addons.mozilla.org/firefox/downloads/file/3588871/xpath_finder-1.0.2-fx.xpi')
        os.makedirs('adds', exist_ok=True)
        with open('adds/xpath.xpi', 'wb') as f:
            f.write(r.content)
        driver.install_addon(os.path.abspath(os.path.join('adds', 'xpath.xpi')), temporary=True)    
    else:
        pass

<br>

## Faz Login

In [ ]:
def get_login(credential):
    # Get Url
    url = 'https://e.ambiente.sp.gov.br/atendimento/'
    driver.get(url)
    time.sleep(5)

    # Hide Popup Tour
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@class="hopscotch-bubble animated tour-portalTour"]')))
    element = driver.find_elements_by_xpath('//*[@class="hopscotch-bubble animated tour-portalTour"]')[1]
    driver.execute_script("arguments[0].style.visibility='hidden'", element)

    try:
        # Efetuar Login
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info-usuario"]'))).click()

        # Pega o XPath do iframe e atribui a uma variável
        iframe = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))

        # Muda o foco para o iframe
        driver.switch_to.frame(iframe)

        # Entra com credenciais
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="j_username"]'))).send_keys(credential['cpf'])
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="j_password"]'))).send_keys(credential['password'])

        # Clica em entrar
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pbEntrar"]'))).click()
    except:
        pass

<br>

## Meus Processos

In [4]:
def get_meusprocessos():
    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Meus Processos"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@title="Meus Processos"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)

    # Pega Table
    table = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="meusProcessos"]')))

    # Linhas
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # Loop
    list_head = []
    list_row = []
    list_processo = []
    for row in rows[:-2]:
        for cel in row.find_elements(By.TAG_NAME, 'th'):        
            list_head.append(cel.text)
        for cel in row.find_elements(By.TAG_NAME, 'td'):
            list_row.append(cel.text)
            if cel.text == 'create_new_folder':
                list_processo.append(cel)

    # Pega Linhas de Interesse
    n_processos = len(rows[:-2])-1
    list_row = [x for x in list_row if x != '']
    list_row = np.array(list_row).reshape(5, n_processos)

    # Dataframe
    df = pd.DataFrame(
        list_row,
        columns=list_head
    )
    df.drop(['Det.'], inplace=True, axis=1)
    return df

<br>

## Processos Liberados

In [5]:
def get_processosliberados():
    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@title="Processos liberados"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)

    # Lista Processos
    tag = '//*[@class="sds-collapse-list sds-card liberacao-collapse-container"]//*[@class="sds-collapse-list__item"]'
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, tag)))
    list_process = driver.find_elements_by_xpath(tag)

    list_process_text = []
    for process in list_process:
        list_process_text.append(process.text.split('\n'))

    # Create Dataframe
    df = pd.DataFrame(
        list_process_text,
        columns=['Nome', '-', 'Data de Liberação', '-', 'Acesso termina em', '-', 'Situação', '-']
    )

    # Edit Dataframe
    df[['Processo','Interessado', 'Empresa']] = df['Nome'].str.split(' - ', expand=True)
    df = df[['Processo','Interessado', 'Empresa', 'Data de Liberação', 'Acesso termina em', 'Situação']].copy()
    df['Interessado'] = df['Interessado'].replace({'Interessado: ': ''}, regex=True)

    # Result
    return df

<br>

## Small Defs

In [6]:
def get_id_liberacao_doc(url):
    return os.path.basename(url)

In [7]:
def select_processo(df_processosliberados):
    # Input
    valores = ''.join('{}'.format(list(df_processosliberados.index)))
    processo = int(input('Insira o número do índice do processo. O número deve estar entre {}: '.format(valores)))
    
    # Check se o processo está no índice
    while not(processo in list(df_processosliberados.index)):
        print('Número não consta no índice. Por favor, insira um número válido!')
        processo = int(input('Insira o número do índice do processo. O número deve estar entre {}: '.format(valores)))
        
    # Check situação
    processo_situacao = df_processosliberados.loc[processo, 'Situação']
    while processo_situacao != 'Acesso ativo':
        processo = int(input('Insira o número do índice do processo. O número deve estar entre {}: '.format(valores)))

    # Result
    processo_text = df_processosliberados.loc[processo, 'Processo'].replace('.', '_').replace('/', '-')
    print('O processo escolhido foi "{}" e está com "{}"'.format(processo_text, processo_situacao))
    return processo, processo_text

<br>

## Get List Files

In [ ]:
def get_file_list(processo):
    # Retorna para a janela principal (fora do iframe)
    driver.switch_to.default_content()

    # Clica em "Processos Liberados"
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@title="Processos liberados"]'))).click()

    # Muda o foco para o iframe
    iframe = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//*[@id="iFrameForm"]')))
    driver.switch_to.frame(iframe)
    time.sleep(2)
    
    # Lista Processos
    tag = '//*[@class="sds-collapse-list sds-card liberacao-collapse-container"]//*[@class="sds-collapse-list__item"]'
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, tag)))
    list_process = driver.find_elements_by_xpath(tag)    

    # Minimiza todos os processos
    for process in list_process:
        try:
            tag = './/button[@class="sds-btn sds-btn--icon button-rotate"]'
            WebDriverWait(process, 1).until(EC.presence_of_element_located((By.XPATH, tag))).click()
            print('a')
        except:
            pass    
    
    # Seleciona Processo    
    WebDriverWait(list_process[processo], 1).until(EC.presence_of_element_located((By.XPATH, './/button[@class="sds-btn sds-btn--icon"]'))).click()
    
    # Lista de todos os arquivos do processo
    tag = '//*[@id="secao-documentos{}"]//*[@class="sds-data-list"]//*[@class="sds-data-list__item"]'.format(processo)
    WebDriverWait(list_process[processo], 3).until(EC.presence_of_element_located((By.XPATH, tag)))
    files = driver.find_elements_by_xpath(tag)

    # Loop
    list_name = []
    list_url = []
    for file in files:
        name = file.text    
        url = file.find_element_by_xpath('.//*[@class="sds-link"]').get_attribute('href')
        list_name.append(name)
        list_url.append(url)

    # Dataframe
    df = pd.DataFrame(
        list(zip(list_name, list_url)),
        columns=['nome','url1']
    )
    df['nome'] = df['nome'].replace({'file_download ': ''}, regex=True)
    df['id_liberacao_doc_1'] = df.apply(lambda x: get_id_liberacao_doc(x['url1']), axis=1)
    return df

<br>

## Get File

<br>

### Função 1: Pega outras URLs

In [ ]:
def get_url_redirects(url):
    # Get URL
    driver.get(url)
    time.sleep(4)
    
    # URL 1
    url1 = driver.current_url
    
    # URL 2    
    url2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]'))).get_attribute('src')
    
    # Results
    urls = list([url1, url2])
    return urls

<br>

### Função 2: Pega Parâmetros

In [ ]:
def adjust_url3(url):
    splitado = url.split('&')

    go_doc_id = splitado[0].split('?')[-1]
    go_doc_id = go_doc_id.split('=')[-1]
    
    id_liberacao_doc = splitado[1]
    id_liberacao_doc = id_liberacao_doc.split('=')[-1]
    
    fl_assinado = splitado[2]
    fl_assinado = fl_assinado.split('=')[-1]
    
    download_url = splitado[3]
    download_url = download_url.split('=')[-1]
    
    return list([go_doc_id, id_liberacao_doc, fl_assinado, download_url])

<br>

### Função 3: Get URL Download

In [8]:
def get_url_download(go_doc_id, id_liberacao_doc):
    # Parametros Link
    dados = {
        'goDocId': go_doc_id,
        'visualizaSemEfeito': 'null',
        'flSituacaoDoc': 'null',
        'flPDF': 'S',
        'idLiberacaoDoc': id_liberacao_doc,
    }

    # URL
    url = ('https://e.ambiente.sp.gov.br/cpa-core-backend/interacao/acesso-externo-documento/documento/?goDocId={}&visualizaSemEfeito={}&flSituacaoDoc={}&flPDF={}&idLiberacaoDoc={}'.format(
        dados['goDocId'],
        dados['visualizaSemEfeito'],
        dados['flSituacaoDoc'],
        dados['flPDF'],
        dados['idLiberacaoDoc'],
    ))
    
    return url

In [ ]:
# Função 1
#url = 'https://e.ambiente.sp.gov.br/cpa-core-backend/interacao/acesso-externo-documento/MjczMzM5'
#urls = get_url_redirects(url)

# Função 2
#url = 'https://e.ambiente.sp.gov.br/go-docs-file-viewer/viewer.html?goDocId=8083252&idLiberacaoDoc=MjczMzQz&flAssinado=N&downloadURL=/cpavPastaExterna/services/pasta-digital/arquivo'
#adjust_url3(url)

# Função 3
#go_doc_id = '7995127'
#id_liberacao_doc = 'MjczMzM5'
#get_url_download(go_doc_id, id_liberacao_doc)

<br>

## Merge PDFs

In [ ]:
def merge_files(processo_text):
    # ddd
    path = os.path.join('data', processo_text)
    filenames = os.listdir(path)
    list_files = [filename for filename in filenames if filename.endswith('.pdf')]

    # Lista documentos da Pasta
    list_dict = []
    for file in list_files:
        n = file.split(' - ', 1)[0]
        list_dict.append({'id': n, 'nomearquivo': file})

    # Dataframe: files
    df = pd.DataFrame(list_dict)
    df['id'] = pd.to_numeric(df['id'])
    df.sort_values(by=['id'], inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'], inplace=True, axis=1)

    # Call the PdfFileMerger
    mergedObject = PdfFileMerger()

    for k, v in df.iterrows():
        bookmark = v['nomearquivo']
        mergedObject.append(
            PdfFileReader(os.path.join(path,  '{}'.format(v['nomearquivo'])), 'rb'),
            bookmark,
        )

    # Write all the files into a file which is named as shown below
    mergedObject.write(os.path.join(path,  '{}.pdf'.format(processo_text)))

<br>

# Rull All

In [ ]:
# Input
credential

# Intro
create_driver()
get_login(credential)

# Meus Processos
#print('Meus Processos')
#df_meusprocessos = get_meusprocessos()
#display(df_meusprocessos)

# Processos Liberados
print('Processos Liberados para Consulta\n')
df_processosliberados = get_processosliberados()
display(df_processosliberados)

# Select Process
processo, processo_text = select_processo(df_processosliberados)

# Get File List
df = get_file_list(processo)
print('\nLista de Arquivos\n...apenas os 5 primeiros apresentados para exemplo..\n')
display(df.head())

# Adjust Table
print('\nObtenção de links para download. Etapa demorada...\n')
df[['url2', 'url3']] = df.apply(lambda x: get_url_redirects(x['url1']), axis=1, result_type='expand')      # Função 1
df[['go_doc_id', 'id_liberacao_doc_2', 'fl_assinado', 'download_url']] = df.apply(lambda x: adjust_url3(x['url3']), axis=1, result_type='expand')    # Função 2
df.loc[:, 'url_download'] = df.apply(lambda x: get_url_download(x['go_doc_id'], x['id_liberacao_doc_1']), axis=1)     # Função 3

# Create Directory
processo_path = os.path.join('data', processo_text)
os.makedirs(processo_path, exist_ok=True)

# Reordena Coluna
df = df[[
    'nome',
    'url1',
    'url2',
    'url3',
    'url_download',
    'go_doc_id',
    'id_liberacao_doc_1',
    'id_liberacao_doc_2',
    'fl_assinado',
    'download_url'
]].copy()

# Escreve Tabela
df.to_csv(
    os.path.join(processo_path, 'tab_arquivos.csv'),
    index=False
)

# Lê Tabela
df = pd.read_csv(
    os.path.join(processo_path, 'tab_arquivos.csv'),
)

# Loop thought table
i = 0
for k, v in df.iterrows():
    r = requests.get(v['url_download'])
    try:
        text = r.content.decode('utf-8')
    except:
        text = 'È PDF!'
    if text != 'Não foi possível recuperar a liberação externa para este documento':
        i += 1        
        print('Download do arquivo "{} - {}"'.format(i, v['nome']))
        filename = os.path.join(processo_path, '{} - {}.pdf'.format(i, v['nome']))
        with open(filename, 'wb') as f:
            f.write(r.content)
    else:
        print('> Erro no download do arquivo "{}"'.format(v['nome']))
        print(v['url_download'])

# Fecha Driver
driver.quit()

# Merge PDF
merge_files(processo_text)

print('Processo "{}" com download completo'.format(processo))